<a href="https://colab.research.google.com/github/aswanthoffl/DDD-Project/blob/main/Model%20Training/Drowsy_eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
cd "/content/drive/MyDrive/EY_Dataset/dataset_new/Eye_dataset/"

/content/drive/MyDrive/EY_Dataset/dataset_new/Eye_dataset


In [ ]:
batchsize=32
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2, validation_split=0.2)
train_data= train_datagen.flow_from_directory(r'train',target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )
validation_data= train_datagen.flow_from_directory(r'train',target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'test',target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 988 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [ ]:
# Load the pre-trained InceptionV3 model without the top layers
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))

# Add new dense layers on top of the pre-trained model
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
        layer.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau,TensorBoard
checkpoint = ModelCheckpoint(r'models/model_eyes1.h5',monitor='val_loss',save_best_only=True,verbose=2)
earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose=2, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 2 )
tensor_board= TensorBoard(r'Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks=[checkpoint,earlystop,learning_rate,tensor_board]

opt = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,validation_data=validation_data,validation_steps=validation_data.samples//batchsize,callbacks=callbacks,epochs=300)

Epoch 1/300
30/30 [==============================] - ETA: 0s - loss: 0.3891 - accuracy: 0.8462 
Epoch 1: val_loss improved from inf to 0.22003, saving model to models/model_eyes1.h5
30/30 [==============================] - 420s 14s/step - loss: 0.3891 - accuracy: 0.8462 - val_loss: 0.2200 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/300
30/30 [==============================] - ETA: 0s - loss: 0.2017 - accuracy: 0.9425
Epoch 2: val_loss improved from 0.22003 to 0.17988, saving model to models/model_eyes1.h5
30/30 [==============================] - 23s 766ms/step - loss: 0.2017 - accuracy: 0.9425 - val_loss: 0.1799 - val_accuracy: 0.9330 - lr: 0.0010
Epoch 3/300
30/30 [==============================] - ETA: 0s - loss: 0.1720 - accuracy: 0.9435
Epoch 3: val_loss improved from 0.17988 to 0.13476, saving model to models/model_eyes1.h5
30/30 [==============================] - 23s 761ms/step - loss: 0.1720 - accuracy: 0.9435 - val_loss: 0.1348 - val_accuracy: 0.9420 - lr: 0.0010
Epoch 4/300
30

In [ ]:
test_accuracy = model.evaluate(test_data, steps=test_data.samples // batchsize)


6/6 [==============================] - 52s 10s/step - loss: 0.0466 - accuracy: 0.9896


In [ ]:
print("Test accuracy:", test_accuracy[1])

Test accuracy: 0.9895833134651184


In [ ]:
'''
adam= 98.4 -------- 98.9
nadam=97.9

'''

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/MyDrive/EY_Dataset/dataset_new/Eye_dataset/models/model_eyes1.h5')

# Define the image size
IMAGE_SIZE = (80, 80)

# Load the image and preprocess it
img = cv2.imread('/content/drive/MyDrive/EY_Dataset/dataset_new/Eye_dataset/test/Closed/_132.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, IMAGE_SIZE)
img = img.reshape(1, 80,80, 3)
img = img / 255.0

# Make the prediction
pred = model.predict(img)

# Print the prediction
if pred[0][0] > pred[0][1]:
    print('Eye is closed')
else:
    print('Eye is open')



# Print the prediction score for each class
print(f'Closed eye score: {pred[0][0]}')
print(f'Open eye score: {pred[0][1]}')

1/1 [==============================] - 1s 1s/step
Eye is closed
Closed eye score: 0.999937891960144
Open eye score: 6.210152059793472e-05


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/EY_Dataset/dataset_new/Eye_dataset/models/model_eyes1.h5')

In [ ]:
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_data)

# Print the validation accuracy
print('Validation accuracy:', val_acc)


8/8 [==============================] - 31s 4s/step - loss: 0.1219 - accuracy: 0.9431
Validation accuracy: 0.9430894255638123
